In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm, multivariate_normal
from joblib import Parallel, delayed

# Load data
data = pd.read_csv('data.csv')

# Constants
num_iterations = 10  # Number of CEM iterations
num_samples = 10_000  # Number of samples per iteration
elite_frac = 0.1  # Fraction of top samples considered elite
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Function to compute loss for a single observation
def compute_loss(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    control_variate = (
        r[0]**0.5 * m_factor
        + (sec_loading - r[0])**0.5 * sec_factor
        + (1 - sec_loading)**0.5 * res_factor
    )
    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = (
            data.loc[ind, 'm'].values
            + data.loc[ind, 'd'].values
            * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    return np.sum(loss)

# Cross-Entropy Method
def cross_entropy_method():
    # Initialize sampling distribution (mean and covariance matrix)
    mean = np.zeros(len(r) + len(data))
    cov = np.eye(len(r) + len(data))

    print("Starting Cross-Entropy Method optimization...")
    for iteration in range(num_iterations):
        # Generate samples from the current distribution
        samples = multivariate_normal.rvs(mean=mean, cov=cov, size=num_samples)

        # Compute losses in parallel
        losses = Parallel(n_jobs=-1)(delayed(compute_loss)(obs) for obs in samples)
        losses = np.array(losses)

        # Select the top elite samples
        elite_threshold = np.percentile(losses, 100 * (1 - elite_frac))
        elite_samples = samples[losses >= elite_threshold]

        # Update the distribution parameters based on elite samples
        mean = np.mean(elite_samples, axis=0)
        cov = np.cov(elite_samples, rowvar=False)

        # Logging
        print(f"Iteration {iteration+1}/{num_iterations}")
        print(f"Elite Threshold Loss: {elite_threshold:.2f}")

    print("Optimization complete. Generating final samples...")
    # Generate final samples from optimized distribution
    final_samples = multivariate_normal.rvs(mean=mean, cov=cov, size=num_samples)
    final_losses = Parallel(n_jobs=-1)(delayed(compute_loss)(obs) for obs in final_samples)
    return np.array(final_losses)

# Run Cross-Entropy Method and Compute VaR
final_losses = cross_entropy_method()

# Compute 99.9% VaR and Variance
VaR_cem = np.percentile(-final_losses, 99.9)
variance_cem = np.var(final_losses)

print(f"\n### Results ###")
print(f"Cross-Entropy Method VaR (99.9%): {VaR_cem:.2f}")
print(f"Cross-Entropy Method Variance: {variance_cem:.2f}")
